## Network creation

Using the transformed data from the previous step, create networks for February and April.

In [1]:
import numpy as np
import networkx as nx
from matplotlib import pyplot as plt
import pickle
from typing import Dict, List, Tuple, Callable, Union

from data_paths import OUT, GRAPHS

Define a seed for random generators.

In [2]:
SEED = 4401

Read transformed data from pickled files.

In [3]:
demographics = pickle.load(open(f'{OUT}demographics.pkl', 'rb'))

comb_counts_pre = pickle.load(open(f'{OUT}comb_counts_pre.pkl', 'rb'))
comb_counts_post = pickle.load(open(f'{OUT}comb_counts_post.pkl', 'rb'))

trip_counts_pre = pickle.load(open(f'{OUT}trip_counts_pre.pkl', 'rb'))
trip_counts_post = pickle.load(open(f'{OUT}trip_counts_post.pkl', 'rb'))

trip_count_change = pickle.load(open(f'{OUT}trip_count_change.pkl', 'rb'))

Store all CBG codes in order.

In [4]:
# store all CBGs in order
ordered_cbgs = sorted(demographics.keys())

Define functions to create an adjacency list between CBGs with transition probabilities as well as a function to compute the cumulative probabilities of the transition probabilities for each CBG.

In [5]:
def create_adjacency_list(comb_counts: Dict[Tuple[str, str], int], 
                          trip_counts: Dict[str, int]) -> Dict[str, List[float]]:
    """
    Create an adjacency list in the form of a dictionary. 
    The keys are the CGBs and the values are ordered lists of transition probabilities to other CBGs.
    The probabilities are in the same order as `ordered_cbgs`.
    The transition probability from CBG i to CBG j is
    P(i, j) = count(trips between i and j) / count(all trips from i to a CBG).
    :param comb_counts: Total count of all trips between two CGBs.
    :param trip_counts: Total count of all trips from each CGB.
    :returns: Adjacency list with probabilites.
    """
    adjacency_list: Dict[str, List[float]] = {}
        
    for i in ordered_cbgs:
        adjacency_list[i] = []
        for j in ordered_cbgs:

            # count of trips between 
            comb = (i, j)
            trips_between = 0 if not comb in comb_counts else comb_counts[comb]

            # ratio of all trips from i
            p = 0 if not i in trip_counts else trips_between / trip_counts[i]
            adjacency_list[i].append(p)
                
    return adjacency_list

def cum_prob_from_adj_list(adjacency_list: Dict[str, List[float]]) -> Dict[str, np.array]:
    """
    From an adjacency list with transition probabilities, calculate the cumulative probabilities.
    :param adjacency_list: Adjacency list with probabilities.
    :returns: Cummulative probabilities for each item in the adjacency list.
    """
    for key in adjacency_list:
        adjacency_list[key] = np.array(adjacency_list[key]).cumsum()

    return adjacency_list

Calculate the adjacency lists.

In [6]:
%%time

adj_list_pre = create_adjacency_list(comb_counts_pre, trip_counts_pre)
adj_list_post = create_adjacency_list(comb_counts_pre, trip_counts_post)

CPU times: user 574 ms, sys: 0 ns, total: 574 ms
Wall time: 581 ms


In [7]:
# sanity check for probabilities
assert 1 - sum(adj_list_pre[ordered_cbgs[0]]) < 0.0001
assert 1 - sum(adj_list_post[ordered_cbgs[0]]) < 0.0001

Calculate the cumulative probabilities.

In [8]:
%%time

cum_prob_pre = cum_prob_from_adj_list(adj_list_pre)
cum_prob_post = cum_prob_from_adj_list(adj_list_post)

CPU times: user 127 ms, sys: 0 ns, total: 127 ms
Wall time: 126 ms


In [9]:
# sanity check for probabilities
assert 1 - cum_prob_pre[ordered_cbgs[0]][-1] < 0.0001
assert 1 - cum_prob_post[ordered_cbgs[0]][-1] < 0.0001

Define distribution functions for different components of the network.

In [10]:
def binary_search(arr, left, right, x) -> int:
    """
    Generic binary search function that finds the element *furthest to the left* of an array.
    :param arr: Array like object.
    :param left: Most left index to start the search.
    :param right: Most right index to start the search.
    :param x: The element to search for.
    :returns: Index of the most left element found found.
    """

    while left < right:
        mid = (left + right) // 2
        if arr[mid] < x:
            left = mid + 1
        else:
            right = mid
    return left

def household_size_distribution(cbg: str, 
                                seed: Union[int, np.random.Generator, None]=None) -> int:
    """
    Household size distribution is drawn from normal distribution with mean 
    according to mean household size of CBG.
    :param seed: random seed.
    :param cbg: CBG of the household.
    :returns: Household size.
    """
    rng = np.random.default_rng(seed=seed)
    mu = demographics[cbg]['household_size']
    # more realistic sd
    sd = mu / 2
    return max(int(rng.normal(mu, sd)), 1)


def household_contact_distribution(baseline: float, cbg: str, multiplier: bool,
                                   seed: Union[int, np.random.Generator, None]=None) -> int:
    """
    Number of connections from a node to another node outside the household.
    :param seed: random seed.
    :param baseline: baseline value for the exponent of the degree distribution.
    :param cbg: CBG of the current household.
    :param multiplier: reduce exponent by factor.
    :returns: Number of connections to outside the household.
    """
    rng = np.random.default_rng(seed=seed)
    if multiplier:
        exponent = baseline * trip_count_change[cbg]
    else:
        exponent = baseline
    return max(int(rng.exponential(exponent)), 1)


def draw_rewire_distribution(cbg: str, cum_prob: Dict[str, np.array],
                             seed: Union[int, np.random.Generator, None]=None) -> str:
    """
    For a given CBG draw from the corresponding rewire distribution and return a random CBG to rewire to.
    :param seed: random seed.
    :param cbg: CBG to be rewired.
    :param cum_prob: The cummulative probability distribution to draw from.
    """
    rng = np.random.default_rng(seed=seed)
    r = rng.random()
    
    # return index where p >= r for the first time
    # idx = next(i for i, p in enumerate(cum_prob[cbg]) if p >= r)
    idx = binary_search(cum_prob[cbg], 0, len(cum_prob[cbg])-1, r)
    return ordered_cbgs[idx]

Define the function to create the network. This is based on `Dobson 2020, Chapter Physical Distancing` but adapted for our purpose.

In [11]:
def create_network(N, cum_prob_rewire: Dict[str, np.array],
                   baseline: float, multiplier: bool=False, 
                   seed: Union[int, np.random.Generator, None]=None) -> nx.Graph:
    """
    Create the network based on some specified distributions.
    :param N: Number of individuals.
    :param cum_prob_rewire: The cummulative probability distribution to draw from for rewiring.
    :param baseline: baseline value for the exponent of the degree distribution.
    :param multiplier: reduce degrees by factor.
    :param seed: seed for random generator.
    :returns: Network.
    """
    
    # initialise variables
    g = nx.Graph()
    rng = np.random.default_rng(seed=seed)
    
    household_id = 1
    households = []
    cbg_id = 1
    
    # total number of nodes in network
    total_node_ct = 0
    cbg_degree_map = {}

    # add the nodes and create the household connections
    for cbg, demographic in demographics.items():

        # target number of nodes for current CBG
        N_cbg = int(demographic['population_prop'] * N)

        # current number of nodes of this CBG
        n = 0
        
        # initialise for later
        cbg_degree_map[cbg] = []

        print(f"Creating nodes: CBG {cbg_id} of {len(demographics)}", end="\r")

        # add households
        while n < N_cbg:

            # create household network
            size = household_size_distribution(cbg, seed=rng)
            house_net = nx.complete_graph(size)

            # add unique labels
            nx.relabel_nodes(house_net, lambda l: total_node_ct + l, copy=False)

            # add nodes and edges of the household to the main network
            g.add_nodes_from(house_net.nodes, household=household_id, household_size=size, cbg=cbg)
            g.add_edges_from(house_net.edges, household=household_id, household_size=size, cbg=cbg)
            households.append(house_net)

            # update iteration values
            n += size
            household_id += 1
            total_node_ct += size

        cbg_id += 1

    print('', end='\n')
            
    # create stubs for connections to outside of household
    stubs = []
    for i in range(len(households)):
        
        print(f"Creating stubs: Household {i+1} of {len(households)}", end="\r")
        
        nodes = list(households[i].nodes)
        
        if len(nodes) > 0:
            cbg = g.nodes[nodes[0]]['cbg']
        
        for node in nodes:
            # draw random degree
            degree = household_contact_distribution(baseline, cbg, multiplier, seed=rng)
            
            # append `degree` number of copies of the current node
            new_stubs = [node] * degree
            stubs.extend(new_stubs)
            cbg_degree_map[cbg].extend(new_stubs)
        
    # add one more if number of stubs is odd
    if len(stubs) % 2:
        unique_stubs = list(set(stubs))
        j = rng.integers(len(unique_stubs))
        
        stubs.append(unique_stubs[j])
        cbg_degree_map[g.nodes[unique_stubs[j]]['cbg']].append(unique_stubs[j])
        
        
    print('', end='\n')
    
    # create pairs of stubs according with rewire distribution
    for i in range(0, len(stubs), 2):
        
        print(f"Creating stub pairs: {i} of {len(stubs)}", end="\r")
        
        while True:
        
            # draw a CBG from the rewire distribution
            cbg = g.nodes[stubs[i]]['cbg']
            target_cbg = draw_rewire_distribution(cbg, cum_prob_rewire, seed=rng)
            
            # make sure the drawn CBG has any available stubs at all
            if len(cbg_degree_map[target_cbg]) == 0:
                continue
                
            # draw a random stub from the required CBG (preserving degree dist.)
            target_node = rng.choice(cbg_degree_map[target_cbg])
            j = stubs.index(target_node)

            # swap nodes
            stubs[i+1], stubs[j] = stubs[j], stubs[i+1]
            break
    
    print('', end='\n')
    
    # Break up intra-household edges resulting from stubs
    while True:
        
        print("Breaking up pairs...", end="\n")    
        
        # iterate by two successive stubs at a time
        swaps = 0
        for i in range(0, len(stubs), 2):
            
            # break up if successive stubs are of same household
            if g.nodes[stubs[i]]['household'] == g.nodes[stubs[i+1]]['household']:
                
                # swap with random other stub
                j = rng.integers(len(stubs))
                stubs[i+1], stubs[j] = stubs[j], stubs[i+1]
                
                swaps += 1
                
        # no swaps, done.
        if swaps == 0:
            break
    
    # connect pairs of stubs
    for i in range(0, len(stubs), 2):
        # label inter-household edges as houshold 0 of size 0
        g.add_edge(stubs[i], stubs[i+1], household=0, household_size=0)

    print('', end='\n')
    
    return (g, [h.order() for h in households])

Create the networks.

In [12]:
%%time
N = 10000
BASELINE = 3
(g_pre, households_pre) = create_network(N=N, cum_prob_rewire=cum_prob_pre, 
                                         baseline=BASELINE, multiplier=False, seed=SEED)

(g_post, households_post) = create_network(N=N, cum_prob_rewire=cum_prob_post, 
                                         baseline=BASELINE, multiplier=True, seed=SEED)

Creating nodes: CBG 628 of 628
Creating stubs: Household 4810 of 4810
Connecting stubs: 28906 of 28908
Breaking up pairs...
Breaking up pairs...

Creating nodes: CBG 628 of 628
Creating stubs: Household 4810 of 4810
Connecting stubs: 14304 of 14306
Breaking up pairs...
Breaking up pairs...

CPU times: user 1min 11s, sys: 1.42 s, total: 1min 12s
Wall time: 1min 11s


Inspect the graph...

In [13]:
print("Pre:")
print('Mean household size of {s:.2f} (range {minf}-{maxf})'.format(s=np.mean(households_pre), minf=min(households_pre), maxf=max(households_pre)))
print('Most connected individual has {k} contacts'.format(k=max(dict(g_pre.degree()).values())))

print("\nPost:")
print('Mean household size of {s:.2f} (range {minf}-{maxf})'.format(s=np.mean(households_post), minf=min(households_post), maxf=max(households_post)))
print('Most connected individual has {k} contacts'.format(k=max(dict(g_post.degree()).values())))

Pre:
Mean household size of 2.12 (range 1-7)
Most connected individual has 28 contacts

Post:
Mean household size of 2.12 (range 1-7)
Most connected individual has 15 contacts


Save the graphs to graphml format.

In [14]:
nx.write_graphml(g_pre, f'{GRAPHS}pre_N_10000.graphml')
nx.write_graphml(g_post, f'{GRAPHS}post_N_10000.graphml')